# Summary of Fraud Detection Code Lab

* Data is highly imbalanced
* Features of interval_after_signup and time related raw and aggregates are highly predictive of fraud
* Made actionable operation recommendations/proposal for business

# Data Exploration

In [1]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import metrics
from sklearn.metrics import roc_curve
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import f1_score, roc_auc_score, roc_curve, precision_recall_curve, auc, make_scorer, recall_score, accuracy_score, precision_score, confusion_matrix
from sklearn.model_selection import GridSearchCV

pd.set_option('display.max_columns', 500)

# Import and store dataset
fraud_data = pd.read_csv('imbalancedFraudDF.csv')
ipToCountry = pd.read_csv('IpAddress_to_Country.csv')

# from sklearn.grid_search import GridSearchCV is deprecated
import sklearn
print(sklearn.__version__)


0.20.2


In [3]:
#Distribution of the label column
fraud_data['class'].value_counts()

0    136961
1      1415
Name: class, dtype: int64

In [4]:
fraud_data.head()

user_id          signup_time        purchase_time  purchase_value  \
0    22058  2015-02-24 22:55:49  2015-04-18 02:47:11              34   
1   333320  2015-06-07 20:39:50  2015-06-08 01:38:54              16   
2   150084  2015-04-28 21:13:25  2015-05-04 13:54:50              44   
3   221365  2015-07-21 07:09:52  2015-09-09 18:40:53              39   
4   159135  2015-05-21 06:03:03  2015-07-09 08:05:14              42   

       device_id source browser sex  age    ip_address  class  
0  QVPSPJUOCKZAR    SEO  Chrome   M   39  7.327584e+08      0  
1  EOGFQPIZPYXFZ    Ads  Chrome   F   53  3.503114e+08      0  
2  ATGTXKYKUDUQN    SEO  Safari   M   41  3.840542e+09      0  
3  NAUITBZFJKHWW    Ads  Safari   M   45  4.155831e+08      0  
4  ALEYXFXINSXLZ    Ads  Chrome   M   18  2.809315e+09      0

In [41]:
# You can install pandas_profiling using the pip package manager by running:
# pip install pandas-profiling

import pandas_profiling

#Inline summary report without saving report as object
pandas_profiling.ProfileReport(fraud_data)

#simpler version without installing pandas_profiling
# fraud_data.describe().transpose()

# will give warnings on missing, correlation, constant value(0 variance), etc, see http://nbviewer.jupyter.org/github/JosPolfliet/pandas-profiling/blob/master/examples/meteorites.ipynb

In [3]:
ipToCountry = pd.read_csv('IpAddress_to_Country.csv')
ipToCountry.head()

lower_bound_ip_address  upper_bound_ip_address    country
0              16777216.0                16777471  Australia
1              16777472.0                16777727      China
2              16777728.0                16778239      China
3              16778240.0                16779263  Australia
4              16779264.0                16781311      China

In [3]:
#Task 1: Identify country info based on ip_address
start = time.time()

countries = []
for i in range(len(fraud_data)):
    ip_address = fraud_data.loc[i, 'ip_address']
    tmp = ipToCountry[(ipToCountry['lower_bound_ip_address'] <= ip_address) &
                    (ipToCountry['upper_bound_ip_address'] >= ip_address)]
    if len(tmp) == 1:#found match
        countries.append(tmp['country'].values[0])
    else:#no match
        countries.append('NA')
        
fraud_data['country'] = countries
print "Lookup took", time.time() - start, "seconds."



Lookup took 209.881716967 seconds.


In [5]:
fraud_data = pd.read_csv('/Users/mingjiang/Documents/lai/fraud/mine/imbalancedFraudDFwithCountry.csv')


## Q: can you optimize the above look-up process?

# Feature Engineering(What types of features can we use?)

## Q: How to check if column user_id is unique(no dup) for time related aggregates?


In [4]:
print fraud_data.user_id.nunique()#138289
print len(fraud_data.index)#138376
#most of the user_id has only the first 1 transaction, difficult to do time based aggregates, 
#e.g. amount/counts in past 1 day for this user

138289
138376


In [6]:
#time related features: can be done before split, as they has no interaction between other rows, solely based on other columns of the same row
fraud_data['interval_after_signup'] = (pd.to_datetime(fraud_data['purchase_time']) - pd.to_datetime(
        fraud_data['signup_time'])).dt.total_seconds()

fraud_data['signup_days_of_year'] = pd.DatetimeIndex(fraud_data['signup_time']).dayofyear

fraud_data['signup_seconds_of_day'] = pd.DatetimeIndex(fraud_data['signup_time']).second + 60 * pd.DatetimeIndex(
    fraud_data['signup_time']).minute + 3600 * pd.DatetimeIndex(fraud_data['signup_time']).hour

fraud_data['purchase_days_of_year'] = pd.DatetimeIndex(fraud_data['purchase_time']).dayofyear
fraud_data['purchase_seconds_of_day'] = pd.DatetimeIndex(fraud_data['purchase_time']).second + 60 * pd.DatetimeIndex(
    fraud_data['purchase_time']).minute + 3600 * pd.DatetimeIndex(fraud_data['purchase_time']).hour

fraud_data = fraud_data.drop(['user_id','signup_time','purchase_time'], axis=1)


In [7]:
fraud_data.head()

purchase_value      device_id source browser sex  age    ip_address  class  \
0              34  QVPSPJUOCKZAR    SEO  Chrome   M   39  7.327584e+08      0   
1              16  EOGFQPIZPYXFZ    Ads  Chrome   F   53  3.503114e+08      0   
2              44  ATGTXKYKUDUQN    SEO  Safari   M   41  3.840542e+09      0   
3              39  NAUITBZFJKHWW    Ads  Safari   M   45  4.155831e+08      0   
4              42  ALEYXFXINSXLZ    Ads  Chrome   M   18  2.809315e+09      0   

         country  interval_after_signup  signup_days_of_year  \
0          Japan              4506682.0                   55   
1  United States                17944.0                  158   
2            NaN               492085.0                  118   
3  United States              4361461.0                  202   
4         Canada              4240931.0                  141   

   signup_seconds_of_day  purchase_days_of_year  purchase_seconds_of_day  
0                  82549                    108                    10031  
1                  74390                    159                     5934  
2                  76405                    124                    50090  
3                  25792                    252                    67253  
4                  21783                    190                    29114

In [10]:
print fraud_data.source.value_counts()

SEO       55747
Ads       54911
Direct    27718
Name: source, dtype: int64


# Q: What now? train/test split? convert categorical features e.g. device_id to numericals? or SMOTE sampling?


In [ ]:
#what is smote
# What if convert source column by OHE now?
# it sneak peek the levels of this column in test data, what if there is another new level "newsPaper" in test, 
# but not in train? info leak, know ahead of time


In [7]:
y = fraud_data['class']
X = fraud_data.drop(['class'], axis=1)

#split into train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print("X_train.shape:", X_train.shape)
print("y_train.shape:", y_train.shape)


('X_train.shape:', (110700, 13))
('y_train.shape:', (110700,))


In [12]:
X_train.head()

purchase_value      device_id  source  browser sex  age    ip_address  \
29343              12  OULPAZAFRFPXP     Ads   Chrome   M   42  3.690922e+09   
12190              10  AIIWMFEYQQIEB     Ads    Opera   M   29  1.686759e+09   
19388              34  VUVETBUPCIWJE  Direct   Chrome   M   53  4.138429e+09   
89104              48  QCFULAJOYKFUU     Ads   Chrome   M   29  9.617337e+07   
82082              44  IHRWLMIJMEEEU     Ads  FireFox   M   24  1.936025e+09   

                 country  interval_after_signup  signup_days_of_year  \
29343  Korea Republic of              3499664.0                  183   
12190      United States              6766039.0                    5   
19388                NaN              5870515.0                  197   
89104             France              2145618.0                  160   
82082              China              7079059.0                  111   

       signup_seconds_of_day  purchase_days_of_year  purchase_seconds_of_day  
29343                  67384                    224                    24648  
12190                  78146                     84                    18585  
19388                  81354                    265                    76669  
89104                  30920                    185                    16538  
82082                  71897                    193                    66156

# Convert categorical features to numericals



In [8]:
#converting needs to be done after split
X_train = pd.get_dummies(X_train, columns=['source', 'browser'])#need to drop ['source', 'browser']? no, auto dropped by get_dummies 
X_train['sex'] = (X_train.sex == 'M').astype(int)

# the more a device is shared, the more suspicious
X_train['n_dev_shared'] = X_train.device_id.map(X_train.device_id.value_counts(dropna=False))

# the more a ip is shared, the more suspicious
X_train['n_ip_shared'] = X_train.ip_address.map(X_train.ip_address.value_counts(dropna=False))

# the less visit from a country, the more suspicious
X_train['n_country_shared'] = X_train.country.map(X_train.country.value_counts(dropna=False))#lots of NAs in country column, #without dropna=False will produce nan in this col

X_train = X_train.drop(['device_id','ip_address','country'], axis=1)



##########################
X_test = pd.get_dummies(X_test, columns=['source', 'browser'])
X_test['sex'] = (X_test.sex == 'M').astype(int)

# the more a device is shared, the more suspicious
X_test['n_dev_shared'] = X_test.device_id.map(X_test.device_id.value_counts(dropna=False))

# the more a ip is shared, the more suspicious
X_test['n_ip_shared'] = X_test.ip_address.map(X_test.ip_address.value_counts(dropna=False))

# the less visit from a country, the more suspicious
X_test['n_country_shared'] = X_test.country.map(X_test.country.value_counts(dropna=False))

X_test = X_test.drop(['device_id','ip_address','country'], axis=1)


In [11]:
X_train.head(20)

purchase_value  sex  age  interval_after_signup  signup_days_of_year  \
29343               12    1   42              3499664.0                  183   
12190               10    1   29              6766039.0                    5   
19388               34    1   53              5870515.0                  197   
89104               48    1   29              2145618.0                  160   
82082               44    1   24              7079059.0                  111   
76812               56    1   25              7872819.0                  102   
111006              67    1   43              7662881.0                  143   
37929               29    0   25              1293152.0                   69   
88089               20    1   18              7551233.0                  225   
50851               14    1   28              6830027.0                  188   
1567                17    0   35              1813911.0                  134   
127834              40    0   45              7893327.0                   41   
84940               77    1   38              5198634.0                  156   
62615               32    1   43              5210838.0                   55   
106910              54    1   33              2301317.0                  126   
109446              38    0   42              6382194.0                   87   
74340               55    0   34               186403.0                   25   
40048               11    1   27              3777778.0                   72   
105255              19    1   33              4258595.0                  147   
77134               38    0   33              4139161.0                  211   

        signup_seconds_of_day  purchase_days_of_year  purchase_seconds_of_day  \
29343                   67384                    224                    24648   
12190                   78146                     84                    18585   
19388                   81354                    265                    76669   
89104                   30920                    185                    16538   
82082                   71897                    193                    66156   
76812                   78778                    194                     2797   
111006                  68977                    232                    42258   
37929                   70051                     84                    67203   
88089                   22512                    312                    56945   
50851                   26963                    267                    31390   
1567                        5                    154                    85916   
127834                  57881                    133                     2408   
84940                   27998                    216                    42632   
62615                   83965                    116                    24403   
106910                  75339                    153                    43856   
109446                  63768                    161                    52362   
74340                    3942                     27                    17545   
40048                   18835                    115                    81413   
105255                  14867                    196                    39862   
77134                   82174                    259                    74135   

        source_Ads  source_Direct  source_SEO  browser_Chrome  \
29343            1              0           0               1   
12190            1              0           0               0   
19388            0              1           0               1   
89104            1              0           0               1   
82082            1              0           0               0   
76812            1              0           0               1   
111006           1              0           0               0   
37929            0              0           1               0   
88089            0              0       

# Q: What is the difference between normalization and standardization?


In [10]:
# # normalize to [0,1], standardize(StandardScaler) to normal, mu=0,var = 1 can < 0, so we do normalize here

# needs to be brought to the same scale for models like LR with regularization(that are not tree based)

#Compute the train minimum and maximum to be used for later scaling:
scaler = preprocessing.MinMaxScaler().fit(X_train[['n_dev_shared', 'n_ip_shared', 'n_country_shared']]) 
#print(scaler.data_max_)

#transform the training data and use them for the model training
X_train[['n_dev_shared', 'n_ip_shared', 'n_country_shared']] = scaler.transform(X_train[['n_dev_shared', 'n_ip_shared', 'n_country_shared']])

#before the prediction of the test data, apply the same scaler obtained from above on X_test, not fitting a brandnew scaler on test
X_test[['n_dev_shared', 'n_ip_shared', 'n_country_shared']] = scaler.transform(X_test[['n_dev_shared', 'n_ip_shared', 'n_country_shared']])


/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [11]:
X_train.n_dev_shared.value_counts(dropna=False)

0.0    105373
0.2      4756
0.4       375
0.6       132
0.8        40
1.0        24
Name: n_dev_shared, dtype: int64

In [12]:
X_test.n_dev_shared.value_counts(dropna=False)

0.0    27310
0.2      350
0.4       12
0.6        4
Name: n_dev_shared, dtype: int64

# Simple LogisticRegression model

In [13]:
# instantiate the model (using the default parameters)
logreg = LogisticRegression()

# fit the model with data
logreg.fit(X_train,y_train)

# predict on test
y_pred=logreg.predict(X_test)

/anaconda2/lib/python2.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [14]:
# import the metrics class
from sklearn import metrics
cm = metrics.confusion_matrix(y_test, y_pred)
cmDF = pd.DataFrame(cm, columns=['pred_0', 'pred_1'], index=['true_0', 'true_1'])
print(cmDF)

        pred_0  pred_1
true_0   27389       0
true_1     287       0


# Simple RF model

In [12]:
classifier_RF = RandomForestClassifier(random_state=0)

classifier_RF.fit(X_train, y_train)

# predict class labels 0/1 for the test set
predicted = classifier_RF.predict(X_test)

# generate class probabilities
probs = classifier_RF.predict_proba(X_test)

# generate evaluation metrics
print("%s: %r" % ("accuracy_score is: ", accuracy_score(y_test, predicted)))
print("%s: %r" % ("roc_auc_score is: ", roc_auc_score(y_test, probs[:, 1])))
print("%s: %r" % ("f1_score is: ", f1_score(y_test, predicted )))#string to int

print ("confusion_matrix is: ")
cm = confusion_matrix(y_test, predicted)
cmDF = pd.DataFrame(cm, columns=['pred_0', 'pred_1'], index=['true_0', 'true_1'])
print(cmDF)
print 'recall =',float(cm[1,1])/(cm[1,0]+cm[1,1])
print 'precision =', float(cm[1,1])/(cm[1,1] + cm[0,1])#1.0

/anaconda2/lib/python2.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


accuracy_score is: : 0.9957002457002457
roc_auc_score is: : 0.8052619232294355
f1_score is: : 0.7384615384615384
confusion_matrix is: 
        pred_0  pred_1
true_0   27389       0
true_1     119     168
recall = 0.585365853659
precision = 1.0


# SMOTE sampling

In [14]:

#Wrong way to sampling: smote before split train/test, then test data does not reflect true distribution in reality, 
#and “bleed” information from the test set into the training of the model. overfit! think about the case of simple 
#oversampling (where I just duplicate observations). If I upsample a dataset before splitting it into a train and 
#validation set, I could end up with the same observation in both datasets

#https://imbalanced-learn.org/en/stable/install.html

# Install
# imbalanced-learn is currently available on the PyPi’s reporitories and you can install it via pip:

# pip install -U imbalanced-learn

#oversampling on only the training data, the right way! 
#sampling_strategy = number of samples in the majority class over the number of samples in the minority class after resampling

smote = SMOTE(random_state=12)
x_train_sm, y_train_sm = smote.fit_sample(X_train, y_train)

unique, counts = np.unique(y_train_sm, return_counts=True)

print np.asarray((unique, counts)).T

[[     0 109572]
 [     1 109572]]


In [15]:
#RF on smoted training data
classifier_RF_sm = RandomForestClassifier(random_state=0)

classifier_RF_sm.fit(x_train_sm, y_train_sm)

# predict class labels for the test set
predicted_sm = classifier_RF_sm.predict(X_test)

# generate class probabilities
probs_sm = classifier_RF_sm.predict_proba(X_test)


# generate evaluation metrics
print("%s: %r" % ("accuracy_score_sm is: ", accuracy_score(y_test, predicted_sm)))
print("%s: %r" % ("roc_auc_score_sm is: ", roc_auc_score(y_test, probs_sm[:, 1])))
print("%s: %r" % ("f1_score_sm is: ", f1_score(y_test, predicted_sm )))#string to int

print ("confusion_matrix_sm is: ")
cm_sm = confusion_matrix(y_test, predicted_sm)
cmDF = pd.DataFrame(cm_sm, columns=['pred_0', 'pred_1'], index=['true_0', 'true_1'])
print(cmDF)
print 'recall or sens_sm =',float(cm_sm[1,1])/(cm_sm[1,0]+cm_sm[1,1])
print 'precision_sm =', float(cm_sm[1,1])/(cm_sm[1,1] + cm_sm[0,1])

#different AUC??

accuracy_score_sm is: : 0.995085995085995
roc_auc_score_sm is: : 0.8022659215028591
f1_score_sm is: : 0.6909090909090909
confusion_matrix_sm is: 
        pred_0  pred_1
true_0   27388       1
true_1     135     152
recall or sens_sm = 0.529616724739
precision_sm = 0.993464052288


# Parameter tuning by GridSearchCV

## Eval metrics for GridSearchCV over all fits upon combination of parameters and cv

In [19]:
# Eval metrics to be calculated for each combination of parameters and cv
scorers = {
    'precision_score': make_scorer(precision_score),
    'recall_score': make_scorer(recall_score),
    'f1_score': make_scorer(f1_score, pos_label=1)
}

In [24]:
def grid_search_wrapper(model, parameters, refit_score='f1_score'):
    """
    fits a GridSearchCV classifier using refit_score for optimization(refit on the best model according to refit_score)
    prints classifier performance metrics
    """
#     skf = StratifiedKFold(n_splits=10)
#     grid_search = GridSearchCV(clf, param_grid, scoring=scorers, refit=refit_score,
#                            cv=skf, return_train_score=True, n_jobs=-1)
    grid_search = GridSearchCV(model, parameters, scoring=scorers, refit=refit_score,
                           cv=3, return_train_score=True, n_jobs=-1)
    grid_search.fit(X_train, y_train)

    # make the predictions
    y_pred = grid_search.predict(X_test)
    y_prob = grid_search.predict_proba(X_test)[:, 1]
    
    print('Best params for {}'.format(refit_score))
    print(grid_search.best_params_)

    # confusion matrix on the test data.
    print('\nConfusion matrix of Random Forest optimized for {} on the test data:'.format(refit_score))
    cm = confusion_matrix(y_test, y_pred)
    cmDF = pd.DataFrame(cm, columns=['pred_0', 'pred_1'], index=['true_0', 'true_1'])
    print(cmDF)
    
    print("\t%s: %r" % ("roc_auc_score is: ", roc_auc_score(y_test, y_prob)))
    print("\t%s: %r" % ("f1_score is: ", f1_score(y_test, y_pred)))#string to int

    print 'recall = ', float(cm[1,1]) / (cm[1,0] + cm[1,1])
    print 'precision = ', float(cm[1,1]) / (cm[1, 1] + cm[0,1])

    return grid_search


## Optimizing on f1_score on LR

In [25]:
# C: inverse of regularization strength, smaller values specify stronger regularization
LRGrid = {"C" : np.logspace(-2,2,5), "penalty":["l1","l2"]}# l1 lasso l2 ridge
#param_grid = {'C': [0.01, 0.1, 1, 10, 100], 'penalty': ['l1', 'l2']}
logRegModel = LogisticRegression(random_state=0)

grid_search_LR_f1 = grid_search_wrapper(logRegModel, LRGrid, refit_score='f1_score')

Best params for f1_score
{'penalty': 'l1', 'C': 0.1}

Confusion matrix of Random Forest optimized for f1_score on the test data:
        pred_0  pred_1
true_0   27387       2
true_1     275      12
	roc_auc_score is: : 0.7638745329103483
	f1_score is: : 0.07973421926910298
recall =  0.0418118466899
precision =  0.857142857143


## Optimizing on f1_score on RF

In [26]:
parameters = {        
'max_depth': [None, 5, 15],
'n_estimators' :  [10,150],
'class_weight' : [{0: 1, 1: w} for w in [0.2, 1, 100]]
}

clf = RandomForestClassifier(random_state=0)

In [24]:
grid_search_rf_f1 = grid_search_wrapper(clf, parameters, refit_score='f1_score')

Best params for f1_score
{'n_estimators': 150, 'max_depth': None, 'class_weight': {0: 1, 1: 0.2}}

Confusion matrix of Random Forest optimized for f1_score on the test data:
   pred_0  pred_1
0   27389       0
1     117     170
	roc_auc_score is: : 0.8081450843143492
	f1_score is: : 0.7439824945295405
recall =  0.592334494774
precision =  1.0


In [25]:
best_rf_model_f1 = grid_search_rf_f1.best_estimator_
best_rf_model_f1

RandomForestClassifier(bootstrap=True, class_weight={0: 1, 1: 0.2},
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=150, n_jobs=None, oob_score=False, random_state=0,
            verbose=0, warm_start=False)

In [37]:
results_f1 = pd.DataFrame(grid_search_rf_f1.cv_results_)
results_sortf1 = results_f1.sort_values(by='mean_test_f1_score', ascending=False)
results_sortf1[['mean_test_precision_score', 'mean_test_recall_score', 'mean_test_f1_score', 'mean_train_precision_score', 'mean_train_recall_score', 'mean_train_f1_score','param_max_depth', 'param_class_weight', 'param_n_estimators']].round(3).head()



mean_test_precision_score  mean_test_recall_score  mean_test_f1_score  \
1                      1.000                   0.577               0.732   
5                      1.000                   0.575               0.730   
0                      0.994                   0.577               0.730   
7                      0.998                   0.575               0.730   
4                      0.992                   0.576               0.729   

   mean_train_precision_score  mean_train_recall_score  mean_train_f1_score  \
1                         1.0                    1.000                1.000   
5                         1.0                    0.614                0.761   
0                         1.0                    0.877                0.935   
7                         1.0                    1.000                1.000   
4                         1.0                    0.638                0.779   

  param_max_depth param_class_weight param_n_estimators  
1            None     {0: 1, 1: 0.2}                150  
5              15     {0: 1, 1: 0.2}                150  
0            None     {0: 1, 1: 0.2}                 10  
7            None       {0: 1, 1: 1}                150  
4              15     {0: 1, 1: 0.2}                 10

In [28]:
#Var Importance, make a guess?
pd.DataFrame(best_rf_model_f1.feature_importances_, index = X_train.columns, columns=['importance']).sort_values('importance', ascending=False)



importance
interval_after_signup      0.405883
purchase_days_of_year      0.146444
signup_seconds_of_day      0.072164
purchase_seconds_of_day    0.070753
signup_days_of_year        0.061146
n_ip_shared                0.060882
n_dev_shared               0.040845
purchase_value             0.038808
age                        0.033563
n_country_shared           0.026520
sex                        0.007915
browser_Chrome             0.006109
browser_IE                 0.005220
source_Ads                 0.005099
source_SEO                 0.005074
source_Direct              0.004438
browser_Safari             0.004238
browser_FireFox            0.004108
browser_Opera              0.000791

## Optimizing  recall_score on RF

In [40]:
grid_search_rf_recall = grid_search_wrapper(clf, parameters, refit_score='recall_score')

Best params for recall_score
{'n_estimators': 150, 'max_depth': 5, 'class_weight': {0: 1, 1: 100}}

Confusion matrix of Random Forest optimized for recall_score on the test data:
   pred_0  pred_1
0   27137     252
1     123     164
	roc_auc_score is: : 0.8152250394783226
	f1_score is: : 0.4665718349928876
recall =  0.571428571429
precision =  0.394230769231


In [39]:
results_recall = pd.DataFrame(grid_search_rf_recall.cv_results_)
results_sortrecall = results_recall.sort_values(by='mean_test_recall_score', ascending=False)
# results_sortf1 = results_f1.sort_values(by='mean_test_f1_score', ascending=False)
results_sortrecall[['mean_test_precision_score', 'mean_test_recall_score', 'mean_test_f1_score', 'mean_train_precision_score', 'mean_train_recall_score', 'mean_train_f1_score','param_max_depth', 'param_class_weight', 'param_n_estimators']].round(3).head()
#recall is worse than default rf?? no this is on test, but train recall is better

mean_test_precision_score  mean_test_recall_score  mean_test_f1_score  \
15                      0.166                   0.662               0.265   
14                      0.193                   0.650               0.295   
0                       0.994                   0.577               0.730   
1                       1.000                   0.577               0.732   
4                       0.992                   0.576               0.729   

    mean_train_precision_score  mean_train_recall_score  mean_train_f1_score  \
15                       0.170                    0.679                0.271   
14                       0.195                    0.663                0.299   
0                        1.000                    0.877                0.935   
1                        1.000                    1.000                1.000   
4                        1.000                    0.638                0.779   

   param_max_depth param_class_weight param_n_estimators  
15               5     {0: 1, 1: 100}                150  
14               5     {0: 1, 1: 100}                 10  
0             None     {0: 1, 1: 0.2}                 10  
1             None     {0: 1, 1: 0.2}                150  
4               15     {0: 1, 1: 0.2}                 10

# Project tasks

In [29]:
#for task 3, based on the above var importance
trainDF = pd.concat([X_train, y_train], axis=1)
pd.crosstab(trainDF["n_dev_shared"],trainDF["class"])
#the larger n_dev_shared, the higher rate of fraud

class              0    1
n_dev_shared             
0.0           104960  413
0.2             4397  359
0.4              167  208
0.6               35   97
0.8                9   31
1.0                4   20

In [34]:
fraud_data.groupby("class")[['interval_after_signup']].mean()#action velocity(consecutive operations/actions of user)
#interval_after_signup on frauds are significantly lower compared to legits

interval_after_signup
class                       
0               5.191179e+06
1               2.354059e+06

In [35]:
fraud_data.groupby("class")[['interval_after_signup']].median()#1
#more than half of fraud happened 1s after signed up

interval_after_signup
class                       
0                  5194911.0
1                        1.0

In [85]:
fraud_data[fraud_data['class'] == 1].head(100)

user_id          signup_time        purchase_time  purchase_value  \
136961    86574  2015-01-11 14:39:04  2015-01-11 14:39:05              61   
136962    74145  2015-03-27 07:55:56  2015-07-20 06:59:34              55   
136963   282058  2015-01-05 08:18:00  2015-01-05 08:18:01              36   
136964   255332  2015-01-01 01:46:58  2015-01-01 01:46:59              19   
136965   399497  2015-01-01 14:29:27  2015-01-22 15:14:12              52   
136966   337297  2015-02-10 13:01:00  2015-04-14 05:08:57              50   
136967    46163  2015-03-05 09:23:48  2015-06-03 01:57:00              12   
136968    17097  2015-01-08 10:34:16  2015-01-08 10:34:17              12   
136969   318808  2015-01-02 06:01:09  2015-01-02 06:01:10              22   
136970   303431  2015-01-11 19:16:05  2015-01-11 19:16:06              61   
136971    44914  2015-01-27 00:43:57  2015-05-24 20:19:06              15   
136972    38019  2015-01-04 22:16:54  2015-01-04 22:16:55              35   
136973   320606  2015-01-10 02:49:15  2015-01-10 02:49:16              40   
136974   186868  2015-01-17 23:28:39  2015-03-10 17:36:30              14   
136975    82337  2015-04-01 01:23:32  2015-06-15 07:15:50              24   
136976    69350  2015-01-01 12:32:04  2015-01-01 12:32:05              36   
136977    77059  2015-07-13 16:30:04  2015-10-12 15:16:24              33   
136978   217420  2015-01-12 10:50:01  2015-01-12 10:50:02              23   
136979    18750  2015-03-01 08:23:07  2015-05-08 23:36:33              23   
136980   214817  2015-06-10 23:25:04  2015-06-11 13:21:31              28   
136981   118283  2015-07-31 13:38:14  2015-09-10 03:34:08              11   
136982   360095  2015-01-09 13:03:19  2015-01-09 13:03:20              42   
136983    69719  2015-01-11 22:10:47  2015-01-11 22:10:48              15   
136984   304193  2015-01-06 20:31:50  2015-01-06 20:31:51              35   
136985    85022  2015-03-12 17:20:23  2015-06-08 05:09:36              41   
136986   254013  2015-05-17 07:10:00  2015-07-13 04:54:28              58   
136987   362057  2015-04-21 23:46:50  2015-07-24 08:45:53              14   
136988   254715  2015-01-04 21:31:00  2015-01-04 21:31:01              31   
136989   147212  2015-01-09 14:16:09  2015-01-09 14:16:10              43   
136990   127514  2015-02-04 18:34:11  2015-03-28 09:00:08              27   
...         ...                  ...                  ...             ...   
137031   364465  2015-01-06 16:33:14  2015-01-06 16:33:15              62   
137032    62189  2015-01-05 14:07:36  2015-01-05 14:07:37              32   
137033   300457  2015-01-07 08:15:54  2015-01-07 08:15:55              65   
137034   281651  2015-05-16 12:23:32  2015-06-16 23:22:37              39   
137035   396318  2015-01-02 02:02:04  2015-01-02 02:02:05               9   
137036    51991  2015-01-07 02:03:20  2015-01-07 02:03:21              15   
137037   172378  2015-03-04 17:31:56  2015-06-08 12:40:57              19   
137038    83058  2015-01-07 17:36:36  2015-01-07 17:36:37               9   
137039    59613  2015-06-05 10:15:33  2015-09-23 02:09:23              47   
137040    54353  2015-08-13 19:51:19  2015-09-04 08:28:49              23   
137041   118764  2015-01-11 09:16:02  2015-01-11 09:16:03              42   
137042     4655  2015-02-25 15:56:46  2015-03-25 10:17:45              12   
137043   104465  2015-01-02 11:44:23  2015-01-02 11:44:24              12   
137044   261871  2015-08-13 22:45:44  2015-08-30 22:53:59              99   
137045   287593  2015-06-26 08:57:56  2015-06-28 02:54:12              24   
137046    40905  2015-01-08 17:36:52  2015-01-08 17:36:53              24   
137047   157067  2015-01-05 19:06:37  2015-01-05 19:06:38              22   
137048   105393  2015-04-16 06:57:59  2015-05-02 20:02:41              88   
137049    12899  2015-02-05 18:57:08  2015-06-02 06:15:23              46   
137050    13762  2015-01-06 07:00:25  2015-01-06 07:00:26              59   
1370

In [102]:
#for task 4, how to use the prediction:
t = (10 * probsBest_recall[:, 1]).astype(int)
unique, counts = np.unique(t, return_counts=True)

print np.asarray((unique, counts)).T

#recommended:
#green: 1 - 3 pass
#grey: 4-7 need manual investigation
#red: 8,9 decline

[[    1     2]
 [    2 26975]
 [    3   202]
 [    4    81]
 [    5   248]
 [    6     8]
 [    7     6]
 [    8    39]
 [    9   115]]


In [ ]:
#HW: try XGBOOST and compare the performance of RF